In [ ]:
import pandas as pd
from api import get_api
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from tqdm import tqdm

### Cluster의 vrp 정보를 가져온다.

In [ ]:
solution = pd.read_csv('solution_cluster_info.csv')

In [ ]:
solution

### Cluster 정보를 나타낸다.

In [ ]:
spot = pd.read_csv('clusterinfo.csv')
spot

### Cluster된 정보를 가져오는 함수

In [ ]:
def find(cNum):
    cNum = cNum[2:]
    result = pd.DataFrame()
    for i in cNum:
        if(i==50):
            break
        temp = spot[spot['cluster']==i]
        result = pd.concat([result, temp],ignore_index=True,axis=0)
        result = result[['lat','lng']]
    return result

In [ ]:
solutionList = solution.values.tolist()
arrive = {'lat': 35.229546, 'lng' : 126.847516}

In [ ]:
def calcdistance(A,B):
    return abs(A['lat']-B['lat']) + abs(A['lng']-B['lng'])

In [ ]:
def calc_distance(originX,originY,destinationX, destinationY):
    # origin and destination should be coordinate
    url = 'https://apis-navi.kakaomobility.com/v1/directions'
    params = {'origin': str(originX) +','+str(originY),
              'destination': str(destinationX) + ','+str(destinationY)}
    header = {'Authorization': 'KakaoAK ' + get_api()}

    route = requests.get(url, params=params, headers=header).json()
#     print(route)

    if route['routes'][0]['result_code'] == 104:
        return 0
    else:
        return(route['routes'][0]['summary']['duration'])

In [ ]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    max_route_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance/60)
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print('Maximum of the route distances: {}m'.format(max_route_distance/60))

In [ ]:
print('start')

In [ ]:
answer = []


for i in solutionList:
    newDF = pd.DataFrame(find(i))
    if(len(newDF) <= 0):
        continue
    newDF = newDF.append(arrive,ignore_index=True)
    result = []
    for j in range(len(newDF)-1):
        temp = []
        for k in range(len(newDF)):
            distance = calcdistance(newDF.iloc[j],newDF.iloc[k]) * 100000
            temp.append(distance)
        result.append(temp)
    result.append([0]*len(newDF))
    print(len(result),len(result[0]),len(result[len(result)-1]))
    def create_data_model():
        data = {}
        data['distance_matrix'] = result
        data['num_vehicles'] = 3
        data['depot'] = len(result)-1
        return data
    
    # Instantiate the data problem.

    data = create_data_model()
    
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)
    
    
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node] 
    
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        100000,  # no slack
        30000000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    
    solution = routing.SolveWithParameters(search_parameters)
    
    def print_solution_only_num():
        output = []
        for vehicle_id in range(data['num_vehicles']):
            Output = []
            index = routing.Start(vehicle_id)
            while not routing.IsEnd(index):
                Output.append(manager.IndexToNode(index))
                index = solution.Value(routing.NextVar(index))
            Output.append(manager.IndexToNode(index))  
            output.append(Output)
        return output
    
    if solution:
        print_solution(data, manager, routing, solution)
        sol = print_solution_only_num()
        answer.append(sol)
        print(sol)
    else:
        print('No solution found !')

In [ ]:
answerDF = pd.DataFrame(answer)

In [ ]:
answerDF.to_csv('allDF.csv',index=False)